# From Normal to Lognormal: The Convexity Adjustment

This notebook visualizes why we need the $-\frac{1}{2}\sigma^2$ term in the lognormal price model.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, lognorm

## The Problem: Jensen's Inequality

For a random variable $X$, the exponential function is **convex**, so:
$$E[e^X] \geq e^{E[X]}$$

The inequality is strict when $X$ has positive variance. This means:
- If $X \sim N(\mu, \sigma^2)$, then $E[e^X] = e^{\mu + \frac{1}{2}\sigma^2}$, **not** $e^{\mu}$

In [2]:
# Visualize the convexity of exp(x)
x = np.linspace(-2, 2, 100)

plt.figure(figsize=(10, 6))
plt.plot(x, np.exp(x), 'b-', linewidth=2, label=r'$e^x$ (convex function)')

# Show a chord (secant line) between two points
x1, x2 = -1, 1
y1, y2 = np.exp(x1), np.exp(x2)
plt.plot([x1, x2], [y1, y2], 'r--', linewidth=2, label='Chord (secant)')

# Midpoint
x_mid = (x1 + x2) / 2
y_chord = (y1 + y2) / 2  # Average of e^x1 and e^x2
y_curve = np.exp(x_mid)   # e^(average of x1 and x2)

plt.scatter([x_mid], [y_chord], color='red', s=100, zorder=5, label=r'$\frac{1}{2}(e^{x_1} + e^{x_2})$ = E[$e^X$]')
plt.scatter([x_mid], [y_curve], color='blue', s=100, zorder=5, label=r'$e^{\frac{1}{2}(x_1+x_2)}$ = $e^{E[X]}$')

plt.axvline(x=x_mid, color='gray', linestyle=':', alpha=0.5)
plt.annotate('', xy=(x_mid + 0.05, y_chord), xytext=(x_mid + 0.05, y_curve),
            arrowprops=dict(arrowstyle='<->', color='green', lw=2))
plt.text(x_mid + 0.15, (y_chord + y_curve)/2, 'Gap from\nconvexity', fontsize=10, color='green')

plt.xlabel('x', fontsize=12)
plt.ylabel(r'$e^x$', fontsize=12)
plt.title(r"Jensen's Inequality: $E[e^X] > e^{E[X]}$ for convex functions", fontsize=14)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.show()

## Simulation: Comparing $E[e^X]$ vs $e^{E[X]}$

Let's simulate and verify that for $X \sim N(\mu, \sigma^2)$:
$$E[e^X] = e^{\mu + \frac{1}{2}\sigma^2}$$

In [3]:
# Parameters
mu = 0.10      # Mean of X
sigma = 0.30   # Std dev of X
n_simulations = 100000

# Simulate X ~ N(mu, sigma^2)
X = np.random.normal(mu, sigma, n_simulations)

# Calculate e^X
eX = np.exp(X)

# Compare
print("Simulation results:")
print(f"  E[X]     = {np.mean(X):.6f}  (true: {mu})")
print(f"  e^E[X]   = {np.exp(np.mean(X)):.6f}  (true: {np.exp(mu):.6f})")
print(f"  E[e^X]   = {np.mean(eX):.6f}  (true: {np.exp(mu + 0.5*sigma**2):.6f})")
print()
print(f"The gap: E[e^X] - e^E[X] = {np.mean(eX) - np.exp(np.mean(X)):.6f}")
print(f"This equals e^mu * (e^(sigma^2/2) - 1) = {np.exp(mu) * (np.exp(0.5*sigma**2) - 1):.6f}")

## Application to Stock Prices

If log-returns are normal: $\ln(S_T/S_0) \sim N(m, \sigma^2 T)$

Then: $S_T = S_0 \cdot e^{\ln(S_T/S_0)}$

And: $E[S_T] = S_0 \cdot e^{m + \frac{1}{2}\sigma^2 T}$

**If we want** $E[S_T] = S_0 \cdot e^{\alpha T}$ (where $\alpha$ is the expected return), we must set:
$$m = \left(\alpha - \frac{1}{2}\sigma^2\right) T$$

In [4]:
# Stock price simulation
S0 = 100        # Initial price
alpha = 0.10    # Expected return (10%)
sigma = 0.30    # Volatility (30%)
T = 1           # 1 year
n_simulations = 100000

# WRONG: Using alpha*T as the mean of log-returns
log_returns_wrong = np.random.normal(alpha * T, sigma * np.sqrt(T), n_simulations)
S_T_wrong = S0 * np.exp(log_returns_wrong)

# CORRECT: Using (alpha - 0.5*sigma^2)*T as the mean of log-returns
log_returns_correct = np.random.normal((alpha - 0.5*sigma**2) * T, sigma * np.sqrt(T), n_simulations)
S_T_correct = S0 * np.exp(log_returns_correct)

print("Target expected price: S0 * e^(alpha*T) = %.4f" % (S0 * np.exp(alpha * T)))
print()
print("WITHOUT convexity adjustment (mean of log-return = alpha*T):")
print(f"  E[S_T] = {np.mean(S_T_wrong):.4f}")
print(f"  Implied return = {np.log(np.mean(S_T_wrong)/S0)/T:.4f} (should be {alpha})")
print()
print("WITH convexity adjustment (mean of log-return = (alpha - 0.5*sigma^2)*T):")
print(f"  E[S_T] = {np.mean(S_T_correct):.4f}")
print(f"  Implied return = {np.log(np.mean(S_T_correct)/S0)/T:.4f} (should be {alpha})")

In [5]:
# Visualize the two distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Log-returns (Normal)
ax1 = axes[0]
bins = np.linspace(-1, 1.5, 50)
ax1.hist(log_returns_wrong, bins=bins, density=True, alpha=0.5, label=f'Mean = $\\alpha T$ = {alpha*T:.2f}', color='red')
ax1.hist(log_returns_correct, bins=bins, density=True, alpha=0.5, label=f'Mean = $(\\alpha - \\frac{{1}}{{2}}\\sigma^2)T$ = {(alpha-0.5*sigma**2)*T:.4f}', color='blue')
ax1.axvline(x=alpha*T, color='red', linestyle='--', linewidth=2)
ax1.axvline(x=(alpha - 0.5*sigma**2)*T, color='blue', linestyle='--', linewidth=2)
ax1.set_xlabel('Log-return', fontsize=12)
ax1.set_ylabel('Density', fontsize=12)
ax1.set_title('Distribution of Log-Returns (Normal)', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Prices (Lognormal)
ax2 = axes[1]
bins = np.linspace(20, 250, 50)
ax2.hist(S_T_wrong, bins=bins, density=True, alpha=0.5, label=f'E[$S_T$] = {np.mean(S_T_wrong):.2f} (wrong)', color='red')
ax2.hist(S_T_correct, bins=bins, density=True, alpha=0.5, label=f'E[$S_T$] = {np.mean(S_T_correct):.2f} (correct)', color='blue')
ax2.axvline(x=np.mean(S_T_wrong), color='red', linestyle='--', linewidth=2)
ax2.axvline(x=np.mean(S_T_correct), color='blue', linestyle='--', linewidth=2)
ax2.axvline(x=S0*np.exp(alpha*T), color='green', linestyle='-', linewidth=2, label=f'Target: $S_0 e^{{\\alpha T}}$ = {S0*np.exp(alpha*T):.2f}')
ax2.set_xlabel('Stock Price $S_T$', fontsize=12)
ax2.set_ylabel('Density', fontsize=12)
ax2.set_title('Distribution of Stock Prices (Lognormal)', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## The Convexity Adjustment as a Function of Volatility

The adjustment $-\frac{1}{2}\sigma^2$ grows quadratically with volatility.

In [6]:
# Show how the adjustment varies with sigma
sigmas = np.linspace(0, 0.8, 100)
adjustment = -0.5 * sigmas**2

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: The adjustment itself
ax1 = axes[0]
ax1.plot(sigmas * 100, adjustment * 100, 'b-', linewidth=2)
ax1.fill_between(sigmas * 100, adjustment * 100, 0, alpha=0.3)
ax1.set_xlabel('Volatility $\\sigma$ (%)', fontsize=12)
ax1.set_ylabel('Convexity adjustment (%)', fontsize=12)
ax1.set_title('The Convexity Adjustment $-\\frac{1}{2}\\sigma^2$', fontsize=14)
ax1.grid(True, alpha=0.3)

# Add some reference points
for s in [0.2, 0.4, 0.6]:
    adj = -0.5 * s**2
    ax1.scatter([s*100], [adj*100], color='red', s=80, zorder=5)
    ax1.annotate(f'$\\sigma$={s*100:.0f}%: adj={adj*100:.1f}%', 
                xy=(s*100, adj*100), xytext=(s*100+5, adj*100-2),
                fontsize=10)

# Right: Mean of log-return vs expected return
ax2 = axes[1]
alpha = 0.10
ax2.axhline(y=alpha*100, color='green', linestyle='-', linewidth=2, label=f'Expected return $\\alpha$ = {alpha*100}%')
ax2.plot(sigmas * 100, (alpha + adjustment) * 100, 'b-', linewidth=2, label=r'Mean of log-return = $\alpha - \frac{1}{2}\sigma^2$')
ax2.fill_between(sigmas * 100, alpha * 100, (alpha + adjustment) * 100, alpha=0.3, label='Gap (convexity adjustment)')
ax2.set_xlabel('Volatility $\\sigma$ (%)', fontsize=12)
ax2.set_ylabel('Rate (%)', fontsize=12)
ax2.set_title('Expected Return vs Mean of Log-Return', fontsize=14)
ax2.legend(loc='lower left')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(-15, 15)

plt.tight_layout()
plt.show()

## Key Takeaway

| Quantity | Formula |
|----------|--------|
| Mean of log-return | $(\alpha - \delta - \frac{1}{2}\sigma^2)T$ |
| Expected return (continuously compounded) | $\alpha$ |
| Expected capital gain | $\alpha - \delta$ |

The $-\frac{1}{2}\sigma^2$ is **not** a penalty or drag on returns. It's simply an accounting adjustment that ensures the parameter $\alpha$ equals the expected return.

**Without the adjustment:** The parameter in the mean would be hard to interpret.

**With the adjustment:** $\alpha$ directly represents the expected return, making it easy to switch to risk-neutral pricing by setting $\alpha = r$.

In [8]:
# Final visualization: the lognormal distribution and its mean
S0 = 100
alpha = 0.10
sigma = 0.30
T = 1

# Parameters for lognormal
mu_log = np.log(S0) + (alpha - 0.5*sigma**2) * T
sigma_log = sigma * np.sqrt(T)

# Create price range
S = np.linspace(1, 250, 500)

# Lognormal PDF
pdf = lognorm.pdf(S, s=sigma_log, scale=np.exp(mu_log))

# Key statistics
E_S = S0 * np.exp(alpha * T)  # Expected value
median_S = np.exp(mu_log)      # Median (= exp of mean of log)
mode_S = np.exp(mu_log - sigma_log**2)  # Mode

plt.figure(figsize=(12, 6))
plt.plot(S, pdf, 'b-', linewidth=2, label='Lognormal PDF')
plt.fill_between(S, pdf, alpha=0.2)

plt.axvline(x=mode_S, color='purple', linestyle='--', linewidth=2, label=f'Mode = {mode_S:.2f}')
plt.axvline(x=median_S, color='orange', linestyle='--', linewidth=2, label=f'Median = {median_S:.2f}')
plt.axvline(x=E_S, color='red', linestyle='-', linewidth=2, label=f'Mean = {E_S:.2f}')
plt.axvline(x=S0, color='gray', linestyle=':', linewidth=2, label=f'Initial $S_0$ = {S0}')

plt.xlabel('Stock Price $S_T$', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.title(f'Lognormal Distribution ($\\alpha$={alpha*100}%, $\\sigma$={sigma*100}%, T={T} year)', fontsize=14)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.xlim(0, 250)

# Add annotation about the skewness
plt.annotate('Mode < Median < Mean\n(positive skewness)', 
            xy=(E_S, 0.002), xytext=(180, 0.012),
            fontsize=11, 
            arrowprops=dict(arrowstyle='->', color='black'),
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.show()

print(f"Mode = exp(mu - sigma^2) = {mode_S:.4f}")
print(f"Median = exp(mu) = {median_S:.4f}")
print(f"Mean = exp(mu + sigma^2/2) = S0 * exp(alpha*T) = {E_S:.4f}")
print(f"\nThe mean is pulled to the right by the positive skewness (convexity of exp).")